<a href="https://colab.research.google.com/github/RejaulBSSE1324/SPL3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import

In [ ]:
!pip install alphashape
import numpy as np
import alphashape
from shapely.geometry import MultiPolygon, Polygon
import plotly.graph_objects as go
from skimage.measure import approximate_polygon
from scipy.interpolate import CubicSpline
from scipy.spatial.distance import cdist
import numpy as np
from scipy.spatial import cKDTree
import math
import matplotlib.pyplot as plt

# Mount Drive and Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load your .pts data
file_path = '/content/drive/MyDrive/02.building roof lidar/Building Boundary/Toronto/roof/build2.pts'
data = np.loadtxt(file_path, usecols=(0, 1, 2), skiprows=0)

# Extract X, Y, Z coordinates
x, y, z = data[:, 0], data[:, 1], data[:, 2]
points_3d = np.column_stack((x, y, z))
points_2d = np.column_stack((x, y))

print(f"Loaded {len(points_2d)} points")

**--------------------------Visualize 2D-------------------**

In [ ]:
def visualize_2d_points(points_2d, title='2D Points Visualization'):
    """Visualize 2D points using scatter plot"""
    points_2d = np.array(points_2d)
    x = points_2d[:, 0]
    y = points_2d[:, 1]

    trace = go.Scatter(
        x=x, y=y,
        mode='markers',
        marker=dict(size=3, color='blue', opacity=0.8)
    )

    layout = go.Layout(
        title=title,
        xaxis_title='X',
        yaxis_title='Y',
        autosize=True
    )

    fig = go.Figure(data=[trace], layout=layout)
    fig.show()

def visualize_overlay(points_2d, contour, title='Contour Overlay'):
    """Visualize original points with contour overlay"""
    fig = go.Figure()

    # Original building points
    fig.add_trace(go.Scatter(
        x=points_2d[:, 0], y=points_2d[:, 1],
        mode='markers',
        marker=dict(size=2, color='lightgray'),
        name='Building Roof Points'
    ))

    # Contour
    fig.add_trace(go.Scatter(
        x=contour[:, 0], y=contour[:, 1],
        mode='lines+markers',
        marker=dict(size=4, color='red'),
        line=dict(width=2, color='red'),
        name='Extracted Contour'
    ))

    fig.update_layout(
        title=title,
        xaxis_title='X',
        yaxis_title='Y',
        autosize=True
    )
    fig.show()

# Visualize original points
visualize_2d_points(points_2d, 'Original Building Point Cloud')

**Calculate Average Point Spacing (k=80) (Paper: Section 2.1.2 )**

In [ ]:
def calculate_average_point_spacing(points_2d, k=80):
    """
    Calculate average point spacing using k anchor points.
    Paper specifies k=80 for stability.
    """
    points_2d = np.array(points_2d)
    k = min(k, len(points_2d))

    tree = cKDTree(points_2d)

    # Randomly select k anchor points
    indices = np.random.choice(points_2d.shape[0], k, replace=False)
    anchor_points = points_2d[indices]

    # Find nearest neighbor for each anchor (k=2 because first is itself)
    distances, _ = tree.query(anchor_points, k=2)
    nearest_distances = distances[:, 1]

    avg_spacing = np.mean(nearest_distances)
    return avg_spacing

# Calculate average spacing
avg_spacing = calculate_average_point_spacing(points_2d, k=80)
print(f"✓ Average point spacing (d): {avg_spacing:.4f} m")

# Calculate paper-specified parameters
W = 8 * avg_spacing  # Band width (Paper: W = 8*d)
T = 10 * avg_spacing  # Densification threshold (Paper: T = 10*d)

print(f"✓ Band width (W = 8*d): {W:.4f} m")
print(f"✓ Densification threshold (T = 10*d): {T:.4f} m")


**            Single-Direction Banding( Paper: Section 2.1.1) **

In [ ]:
def extract_contour_single_direction(points_2d, band_width, angle=0):
    """
    Extract contour points using single-direction banding.

    Process:
    1. Rotate points to align bands with specified angle
    2. Divide into bands of width W along x-axis
    3. For each band, project points onto central axis
    4. Find two points with maximum and minimum y-coordinates
    """
    # Rotate points
    angle_rad = np.deg2rad(angle)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])

    rotated_points = points_2d @ rotation_matrix.T

    # Create bands along x-axis
    min_x, max_x = rotated_points[:, 0].min(), rotated_points[:, 0].max()
    bands = np.arange(min_x, max_x + band_width, band_width)

    contour_points = []

    for i in range(len(bands) - 1):
        band_left = bands[i]
        band_right = bands[i + 1]

        # Get points in this band
        mask = (rotated_points[:, 0] >= band_left) & (rotated_points[:, 0] < band_right)
        band_points_rotated = rotated_points[mask]

        if len(band_points_rotated) == 0:
            continue

        # Find two points with max and min y-coordinates (extreme points)
        min_idx = np.argmin(band_points_rotated[:, 1])
        max_idx = np.argmax(band_points_rotated[:, 1])

        # Get original point indices and add to contour
        band_indices = np.where(mask)[0]
        contour_points.append(points_2d[band_indices[min_idx]])
        if min_idx != max_idx:
            contour_points.append(points_2d[band_indices[max_idx]])

    return np.array(contour_points) if contour_points else np.empty((0, 2))

# Extract single-direction contour (0 degrees for demonstration)
single_contour = extract_contour_single_direction(points_2d, W, angle=0)
print(f"✓ Single direction (0°): {len(single_contour)} contour points extracted")

visualize_2d_points(single_contour, 'Single Direction Contour (0°)')

Multidirectional Banding (6 directions) Paper: Section 2.1.3 - Use N₀ = 6 directions

In [ ]:
def extract_contour_multidirectional(points_2d, band_width):
    """
    Extract contour using 6 directions as specified in paper.
    Directions: 0°, 30°, 60°, 90°, 120°, 150°

    This ensures complete coverage and prevents missing contour points
    in concave areas.
    """
    directions = [0, 30, 60, 90, 120, 150]  # Paper-specified directions
    all_contour_points = []

    print(f"Extracting contour in {len(directions)} directions...")

    for angle in directions:
        contour = extract_contour_single_direction(points_2d, band_width, angle)
        if len(contour) > 0:
            all_contour_points.append(contour)
            print(f"  {angle}°: {len(contour)} points")

    if not all_contour_points:
        return np.empty((0, 2))

    # Combine all contour points
    all_points = np.vstack(all_contour_points)

    # Remove duplicates (points may be extracted in multiple directions)
    unique_points = np.unique(np.round(all_points, decimals=6), axis=0)

    return unique_points

# Extract multidirectional contour
multi_contour = extract_contour_multidirectional(points_2d, W)
print(f"\n✓ Total unique contour points: {len(multi_contour)}")

visualize_2d_points(multi_contour, 'Multidirectional Contour (6 directions)')

# Optimization & Densification of Contour Points

**Sort Contour Points with β=240° Constraint Paper: Section 2.2.1**

In [ ]:
def sort_contour_points_with_constraint(points, beta_degrees=240):
    """
    Sort contour points by nearest neighbor with forward direction constraint.

    Paper constraint: Use β=240° sector to prevent backward connections.
    This ensures the contour line progresses in a consistent direction.
    """
    if len(points) < 2:
        return points

    sorted_pts = [points[0]]
    remaining = list(range(1, len(points)))
    last_direction = None

    while remaining:
        current = sorted_pts[-1]

        # Calculate vectors and distances to all remaining points
        candidates = points[remaining]
        vectors = candidates - current
        distances = np.linalg.norm(vectors, axis=1)

        if last_direction is not None:
            # Calculate angles relative to last direction
            angles = np.arctan2(vectors[:, 1], vectors[:, 0])
            last_angle = np.arctan2(last_direction[1], last_direction[0])

            # Angle difference (normalized to [0, π])
            angle_diff = np.abs(angles - last_angle)
            angle_diff = np.minimum(angle_diff, 2*np.pi - angle_diff)

            # Filter by β constraint (240° = 4.189 radians)
            beta_rad = np.deg2rad(beta_degrees)
            valid_mask = angle_diff <= beta_rad

            if np.any(valid_mask):
                # Choose nearest valid point
                valid_distances = np.where(valid_mask, distances, np.inf)
                next_idx = np.argmin(valid_distances)
            else:
                # If no valid points, choose nearest
                next_idx = np.argmin(distances)
        else:
            # First connection: just choose nearest
            next_idx = np.argmin(distances)

        # Add next point and update
        next_point_idx = remaining[next_idx]
        sorted_pts.append(points[next_point_idx])
        last_direction = points[next_point_idx] - current
        remaining.pop(next_idx)

    return np.array(sorted_pts)

# Sort contour points
sorted_contour = sort_contour_points_with_constraint(multi_contour, beta_degrees=240)
print(f"✓ Sorted {len(sorted_contour)} contour points with β=240° constraint")

visualize_overlay(points_2d, sorted_contour, 'Sorted Contour ')

**Densify Long Edges (CORRECT PAPER METHOD) Paper: Section 2.2.2**

In [ ]:
def densify_contour_paper_method(sorted_contour, all_points_2d, threshold):
    """
    Densify contour by inserting ACTUAL points from the point cloud.

    CRITICAL DIFFERENCE from your original code:
    - Your code: Creates interpolated (fake) points
    - Paper method: Finds closest REAL points from the original point cloud

    Process:
    1. Find edges longer than threshold T
    2. Calculate midpoint of long edge
    3. Find closest NON-CONTOUR point to midpoint
    4. Insert this point into contour
    5. Repeat until all edges < T
    """
    if len(sorted_contour) < 2:
        return sorted_contour

    densified = list(sorted_contour)
    tree = cKDTree(all_points_2d)

    max_iterations = 100
    iteration = 0

    print("Densifying long edges...")

    while iteration < max_iterations:
        iteration += 1
        changed = False

        i = 0
        while i < len(densified):
            p1 = densified[i]
            p2 = densified[(i + 1) % len(densified)]

            edge_length = np.linalg.norm(p2 - p1)

            if edge_length > threshold:
                # Find midpoint of long edge
                midpoint = (p1 + p2) / 2

                # Find closest points to midpoint from original cloud
                distances, indices = tree.query(midpoint, k=20)

                best_point = None
                best_dist = np.inf

                # Find closest NON-CONTOUR point
                for dist, idx in zip(distances, indices):
                    candidate = all_points_2d[idx]

                    # Check if this point is already in contour
                    is_in_contour = any(
                        np.allclose(candidate, cp, atol=1e-6)
                        for cp in densified
                    )

                    if not is_in_contour and dist < best_dist:
                        best_point = candidate
                        best_dist = dist
                        break

                # Insert the found point
                if best_point is not None:
                    densified.insert(i + 1, best_point)
                    changed = True
                    print(f"  Iteration {iteration}: Inserted point, edge length {edge_length:.4f} → split")
                    break

            i += 1

        if not changed:
            print(f"  Converged after {iteration} iterations")
            break

    return np.array(densified)

# Densify the contour
densified_contour = densify_contour_paper_method(sorted_contour, points_2d, T)
print(f"✓ Densified contour: {len(densified_contour)} points (added {len(densified_contour) - len(sorted_contour)} points)")

visualize_overlay(points_2d, densified_contour, 'Densified Contour')

**Remove Noise Points Paper: Section 2.2.2**

In [ ]:
def remove_noise_points(contour_points, points_3d, avg_spacing, threshold_factor=5):
    """
    Remove noise points based on elevation difference.

    Paper method:
    1. For each contour point, find 5 nearest neighbors
    2. Calculate average elevation of these 5 neighbors
    3. If elevation difference > 5*d, consider it noise
    4. Remove noise points (e.g., parapet walls)
    """
    if len(contour_points) < 6:
        return contour_points

    threshold = threshold_factor * avg_spacing

    # Match contour points with 3D points
    tree_3d = cKDTree(points_3d[:, :2])
    contour_3d = []

    for pt_2d in contour_points:
        _, idx = tree_3d.query(pt_2d, k=1)
        contour_3d.append(points_3d[idx])

    contour_3d = np.array(contour_3d)

    filtered = []
    removed_count = 0

    for i, point in enumerate(contour_3d):
        # Find 5 nearest contour points (in 2D space)
        distances = []
        for j, other in enumerate(contour_3d):
            if i != j:
                dist = np.linalg.norm(point[:2] - other[:2])
                distances.append((dist, j))

        distances.sort()
        nearest_5_indices = [idx for _, idx in distances[:min(5, len(distances))]]

        # Calculate average elevation
        avg_z = np.mean([contour_3d[idx][2] for idx in nearest_5_indices])
        elev_diff = abs(point[2] - avg_z)

        if elev_diff < threshold:
            filtered.append(point[:2])  # Keep only X, Y
        else:
            removed_count += 1

    print(f"✓ Removed {removed_count} noise points (threshold: {threshold:.4f}m)")

    return np.array(filtered) if filtered else contour_points

# Remove noise points
final_contour = remove_noise_points(densified_contour, points_3d, avg_spacing)
print(f"✓ Final contour: {len(final_contour)} points")

visualize_overlay(points_2d, final_contour, 'Final Contour (Noise Removed)')

**Close the contour**

In [ ]:
def close_contour(points):
    """Close the contour by connecting last point to first"""
    if len(points) > 0 and not np.allclose(points[0], points[-1]):
        return np.vstack([points, points[0]])
    return points

final_contour_closed = close_contour(final_contour)
print(f"✓ Contour closed: {len(final_contour_closed)} points")

visualize_overlay(points_2d, final_contour_closed, 'Final Closed Contour')

# Summary Statistics

In [ ]:
print("\n" + "="*60)
print("EXTRACTION SUMMARY")
print("="*60)
print(f"Input points:              {len(points_2d)}")
print(f"Average spacing (d):       {avg_spacing:.4f} m")
print(f"Band width (W = 8*d):      {W:.4f} m")
print(f"Threshold (T = 10*d):      {T:.4f} m")
print(f"\nExtraction stages:")
print(f"  Single direction (0°):   {len(single_contour)} points")
print(f"  Multi-direction (6×):    {len(multi_contour)} points")
print(f"  After sorting:           {len(sorted_contour)} points")
print(f"  After densification:     {len(densified_contour)} points")
print(f"  After noise removal:     {len(final_contour)} points")
print(f"  Final closed contour:    {len(final_contour_closed)} points")
print("="*60)

# Compare Methods

In [ ]:
def visualize_comparison(points_2d, stages):
    """Visualize all extraction stages for comparison"""
    fig = go.Figure()

    # Original points
    fig.add_trace(go.Scatter(
        x=points_2d[:, 0], y=points_2d[:, 1],
        mode='markers',
        marker=dict(size=2, color='lightgray', opacity=0.5),
        name='Original Points'
    ))

    colors = ['orange', 'blue', 'green', 'purple', 'red']
    stage_names = ['Single Dir', 'Multi Dir', 'Sorted', 'Densified', 'Final']

    for i, (stage, color, name) in enumerate(zip(stages, colors, stage_names)):
        if len(stage) > 0:
            fig.add_trace(go.Scatter(
                x=stage[:, 0], y=stage[:, 1],
                mode='lines+markers',
                marker=dict(size=3, color=color),
                line=dict(width=1, color=color),
                name=name,
                visible='legendonly' if i < len(stages)-1 else True
            ))

    fig.update_layout(
        title='Building Contour Extraction - All Stages',
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        hovermode='closest',
        legend=dict(x=0, y=1)
    )

    fig.show()

stages = [single_contour, multi_contour, sorted_contour, densified_contour, final_contour_closed]
visualize_comparison(points_2d, stages)